In [1]:
import numpy as np
from numpy import random
import re
import pickle
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
from pathlib import Path
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

%matplotlib inline
matplotlib.style.use('ggplot')

C:\Users\noor1\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
#import dataset
from dataset import Dataset

from keras import losses, models, optimizers, initializers, regularizers

from keras.models import Sequential, Model, model_from_json

from keras.preprocessing.text import Tokenizer

from keras.layers import (Layer, Input, Flatten, Dropout, BatchNormalization, Reshape, Embedding,
                          MaxPool1D, AveragePooling1D, GlobalAveragePooling1D,
                          Conv1D, SeparableConv1D, Dense, LeakyReLU, ReLU, Activation,
                          LSTM, SimpleRNNCell, Bidirectional)

In [ ]:
#dataset = pd.read_csv("flip.csv", names={'product_name','product_category','description'})
#filename = 'datasets/amazon_co-ecommerce_sample.csv'
#text_field='product_name'

#dataset = Dataset(filename)
#data=pd.read_pickle(text_field=text_field, label_field='product_category', root_label=True)
#dataset.filer_data()
#dataset.text_info()

In [ ]:
filename = 'flip.csv'
text_field='product_name'

#dataset = pd.read_csv("flip.csv", names={'product_name','product_category','description'})
dataset = Dataset(filename)
dataset.load(text_field=text_field, label_field='product_category', root_label=True)
dataset.filer_data()
dataset.text_info()

In [ ]:
tokenizer = dataset.tokenize()
label_encoder = dataset.label_encode()

data_train, data_test = train_test_split(dataset.data, test_size=0.15)
    
token_train = tokenizer.texts_to_sequences(data_train.text)
text_train = pad_sequences(token_train, maxlen=dataset.max_text())
label_train = label_encoder.transform(data_train.label)
        
token_test = tokenizer.texts_to_sequences(data_test.text)
text_test = pad_sequences(token_test, maxlen=dataset.max_text())
label_test = label_encoder.transform(data_test.label)

#dataset=dataset.apply(word_tokenize)
dataset.head()

In [ ]:
train_word2vec = True
if train_word2vec:
    size = 500
    max_vocab_size = 20000

    sentences = dataset.text.transform(lambda t: t.split())

    embedding = Word2Vec(sentences, size=size,
                         max_vocab_size=max_vocab_size)

    output_file = 'embeddings/{}_s{}.txt'.format('word2vec', size)
    embedding.wv.save_word2vec_format(output_file)

In [ ]:
embedding_filename = 'word2vec.ipynb'

embedding = KeyedVectors.load_word2vec_format(embedding_filename)

embedding_matrix = np.zeros((len(word_index) + 1, embedding.vector_size))
for word, i in word_index.items():
    try:
        embedding_vector = embeddings.get_vector(word)
        embedding_matrix[i] = embedding_vector
    except:
        pass


In [ ]:
embedding_layer = Embedding(input_dim=len(word_index) + 1,
                            output_dim=embedding.vector_size,
                            weights=[embedding_matrix],
                            input_length=dataset.max_text(),
                            trainable=True)

In [ ]:
use_sepcnn = True
if use_sepcnn: 
    
    # Hyper Parameters
    blocks       = 1
    filters      = 32
    kernel_size  = 3
    dropout_rate = 0.2
    pool_size    = 3
    
    #input_layer = Input(shape=(dataset.max_text(),), dtype='int32')
    input_layer = tf.keras.Input(shape=(32,),dtype='int32')
    x = embedding_layer(input_layer)

    for _ in range(blocks-1):
        x = Dropout(rate=dropout_rate)(x)   
        x = SeparableConv1D(filters=filters, kernel_size=kernel_size, activation='relu',bias_initializer='random_uniform', depthwise_initializer='random_uniform',padding='same')(x)
        x = SeparableConv1D(filters=filters, kernel_size=kernel_size, activation='relu',bias_initializer='random_uniform', depthwise_initializer='random_uniform',padding='same')(x) 
        x = MaxPool1D(pool_size=pool_size)(x) 
        x = SeparableConv1D(filters=filters * 2, kernel_size=kernel_size, activation='relu',bias_initializer='random_uniform', depthwise_initializer='random_uniform',padding='same')(x) 
        x = SeparableConv1D(filters=filters * 2, kernel_size=kernel_size, activation='relu',bias_initializer='random_uniform', depthwise_initializer='random_uniform',padding='same')(x) 
        x = GlobalAveragePooling1D()(x) 
        x = Dropout(rate=dropout_rate)(x) 
        output_layer = Dense(dataset.label_unique.size, activation='softmax')(x)

    model = Model(input_layer, output_layer)
    model.name = 'sepcnn_' + str(embedding.vector_size) + '_' + text_field
    model.summary()

In [ ]:
use_jatana_lstm = True
if use_jatana_lstm:
    
    # Hyper Parameters
    units       = 100
    drop_ratio  = 0.3
    drop_rec_ratio = 0.3
    
    input_layer = Input(shape=(dataset.max_text(),), dtype='int32')
    x = embedding_layer(input_layer)
    x = Bidirectional(LSTM(units, dropout=drop_ratio, recurrent_dropout=drop_rec_ratio))(x)
    output_layer = Dense(dataset.label_unique.size, activation='softmax')(x)

    model = Model(input_layer, output_layer)
    model.name = 'lstm_jatana_' + str(embedding.vector_size) + '_' + text_field
    model.summary()

In [ ]:
model.save('created_models/'+ model.name + '.h5')

In [ ]:
PATH_MODELS = 'created_models/'
MODEL = PATH_MODELS + 'sepcnn_100_name.h5'

In [ ]:
model = load_model(MODEL)